# Wildcard Agent
Use wildcard to dynamically select the right tool to use based on the user's request.

In [ ]:
%pip install agentsjson openai

### Setup API keys

In [15]:
OPENAI_API_KEY="<your-openai-api-key>"
RESEND_API_KEY="your-resend-api-key"
WILDCARD_API_KEY="your-wildcard-api-key"

### Setup Authentication

In [17]:
from agentsjson.core.models.auth import ApiKeyAuthConfig, AuthType

# API Key config for Resend
resend_auth = ApiKeyAuthConfig(
    type=AuthType.API_KEY,
    key_value=RESEND_API_KEY,
    key_name=None,
    key_prefix=None
)

### Setup your agent Toolkit
If you need to use multiple tools, you can add them to the toolkit.

In [18]:
toolkit = {
    "resend": resend_auth,
}

def find_auth(agents_json):
    api = agents_json.sources[0].id
    return toolkit[api]

### Wildcard Tool Selection
Find the right tool to use for the given task.

Note: For resend you need to first create a valid domain.
You can do this from your resend dashboard. Or you can query the wildcard agent to do this: "Create a new domain in resend called 'wild-card.ai'"

You may need some additional manual steps to add the domain to MX records in your DNS provider, there's no API for this.

In [30]:
query = "Create a new domain in resend called 'wild-card.ai'"


In [31]:
import requests
from agentsjson.core.models.schema import AgentsJson


def find_tool(query):
    result = requests.get(
        "https://api.wild-card.ai/search",
        params={
            "query": query,
            "collection_id": "c58a5915-7b28-49b1-a28d-d81ffc6f63ea"
        },
        headers={"X-API-Key": WILDCARD_API_KEY}
    )
    
    result = result.json()
    if result.get("error"):
        raise ValueError("Error: ", result.get("error"))
    
    return AgentsJson(**result)

In [ ]:
agents_json = find_tool(query)
agents_json

### Add the result to LLM to generate arguments

In [38]:
from openai import OpenAI
from agentsjson.core.models.auth import BearerAuthConfig, AuthType
from agentsjson import ToolFormat, execute, get_tools

def run_agent(query, agents_json):
    # Get the tool definition for the selected action
    tools = get_tools(agents_json, ToolFormat.OPENAI)

    client = OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {
            "role": "system", 
            "content": "You are a helpful assistant that completes tasks."
        },
        {
            "role": "user", 
            "content": query
        }],
        tools=tools
    )
    return response


In [ ]:
response = run_agent(query)
response

### Execute the chosen action with authentication

In [ ]:
# Execute the chosen action with authentication

auth = find_auth(agents_json)
result = execute(agents_json, response, ToolFormat.OPENAI, auth)
result

### Try another query

In [ ]:
query2 = "Send an email to kaushik@wild-card.ai from contact@wild-card.ai with the subject 'Hello from Wild Card'"


agents_json2 = find_tool(query2)
response2 = run_agent(query2, agents_json2)
auth2 = find_auth(agents_json2)
result2 = execute(agents_json2, response2, ToolFormat.OPENAI, auth2)
result2
